In [8]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from datetime import datetime, date, timedelta
import os
import xarray as xr

In [16]:
df = pd.read_csv("../data/processed/predictors/formation.csv", parse_dates=["formation_datetime"])
df = df[df.formation_datetime.dt.year >= 1940]

In [24]:
ds = nc.Dataset('../data/raw/cape.nc')
print(ds.variables['var59'].get_fill_value())

None


In [17]:
def get_cape_by_date_offset(ts, lat, lon, days_earlier):
	form_date = ts.date() # Convert from Timestamp to datetime.date
	target_date = form_date - timedelta(days = days_earlier)
	# target_prev_sunday = target_date - timedelta(days = (target_date.weekday() + 1) % 7)
	# target_first_of_month = date(target_date.year, target_date.month, 1)

	path = "cape.nc"

	with xr.open_dataset(os.path.join("../data", "raw", path)) as ds:
		cape = ds['var59'].sel(
			lat = lat,
			lon = lon,
			time = target_date,
			method = 'nearest'
		).item()

	return cape

In [18]:
lead_times = [0, 2, 3]
for lead in lead_times:
	df[f'cape_minus_{lead}_days'] = df.apply(axis='columns', func = lambda row: get_cape_by_date_offset(row['formation_datetime'], row['formation_lat'], row['formation_lon'], lead))
df.head()

,code,formation_datetime,formation_lat,formation_lon,cape_minus_0_days,cape_minus_2_days,cape_minus_3_days
89,AL011940,1940-05-19 12:00:00,22.0,289.0,0.000,3.500,0.00
90,AL011941,1941-09-11 00:00:00,27.1,273.3,178.125,196.875,390.00
91,AL011942,1942-08-03 00:00:00,16.2,274.8,399.250,517.500,517.25
92,AL011943,1943-07-25 18:00:00,28.0,272.3,1.250,0.375,0.50
93,AL011944,1944-07-13 06:00:00,19.6,291.4,10.125,15.500,0.00


In [19]:
results = df.drop(axis='columns', labels=['formation_lat','formation_lon','formation_datetime'])
results.head()

,code,cape_minus_0_days,cape_minus_2_days,cape_minus_3_days
89,AL011940,0.000,3.500,0.00
90,AL011941,178.125,196.875,390.00
91,AL011942,399.250,517.500,517.25
92,AL011943,1.250,0.375,0.50
93,AL011944,10.125,15.500,0.00


In [21]:
results.to_csv('../data/processed/predictors/cape.csv', index=False)